# 多模态 RAG 与图像描述 (Multi-Modal RAG with Image Captioning)

在这个笔记本中，我实现了一个多模态 RAG 系统，该系统从文档中提取文本和图像，为图像生成描述，并使用两种内容类型来响应查询。这种方法通过将视觉信息纳入知识库来增强传统的 RAG。

传统的 RAG 系统只处理文本，但许多文档在图像、图表和表格中包含关键信息。通过为这些视觉元素生成描述并将它们纳入我们的检索系统，我们可以：

- 访问锁定在图形和图表中的信息
- 理解补充文本的表格和图表
- 创建更全面的知识库
- 回答依赖于视觉数据的问题

## 设置环境
我们首先导入必要的库。

In [1]:
import os
import io
import numpy as np
import json
import fitz
from PIL import Image
from openai import OpenAI
import base64
import re
import tempfile
import shutil

## 设置 OpenAI API 客户端
我们初始化 OpenAI 客户端来生成嵌入向量和响应。

In [ ]:
# 使用基础 URL 和 API 密钥初始化 OpenAI 客户端
client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.getenv("OPENAI_API_KEY")  # 从环境变量中获取 API 密钥
)

## 文档处理函数

In [3]:
def extract_content_from_pdf(pdf_path, output_dir=None):
    """
    从 PDF 文件中提取文本和图像。
    
    参数:
        pdf_path (str): PDF 文件的路径
        output_dir (str, optional): 保存提取图像的目录
        
    返回:
        Tuple[List[Dict], List[Dict]]: 文本数据和图像数据
    """
    # 如果未提供，为图像创建临时目录
    temp_dir = None
    if output_dir is None:
        temp_dir = tempfile.mkdtemp()
        output_dir = temp_dir
    else:
        os.makedirs(output_dir, exist_ok=True)
        
    text_data = []  # 存储提取的文本数据的列表
    image_paths = []  # 存储提取的图像路径的列表
    
    print(f"从 {pdf_path} 提取内容...")
    
    try:
        with fitz.open(pdf_path) as pdf_file:
            # 遍历 PDF 中的每一页
            for page_number in range(len(pdf_file)):
                page = pdf_file[page_number]
                
                # 从页面提取文本
                text = page.get_text().strip()
                if text:
                    text_data.append({
                        "content": text,
                        "metadata": {
                            "source": pdf_path,
                            "page": page_number + 1,
                            "type": "text"
                        }
                    })
                
                # 从页面提取图像
                image_list = page.get_images(full=True)
                for img_index, img in enumerate(image_list):
                    xref = img[0]  # 图像的 XREF
                    base_image = pdf_file.extract_image(xref)
                    
                    if base_image:
                        image_bytes = base_image["image"]
                        image_ext = base_image["ext"]
                        
                        # 将图像保存到输出目录
                        img_filename = f"page_{page_number+1}_img_{img_index+1}.{image_ext}"
                        img_path = os.path.join(output_dir, img_filename)
                        
                        with open(img_path, "wb") as img_file:
                            img_file.write(image_bytes)
                        
                        image_paths.append({
                            "path": img_path,
                            "metadata": {
                                "source": pdf_path,
                                "page": page_number + 1,
                                "image_index": img_index + 1,
                                "type": "image"
                            }
                        })
        
        print(f"提取了 {len(text_data)} 个文本段落和 {len(image_paths)} 张图像")
        return text_data, image_paths
    
    except Exception as e:
        print(f"提取内容时出错：{e}")
        if temp_dir and os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)
        raise

## 文本内容分块

In [4]:
def chunk_text(text_data, chunk_size=1000, overlap=200):
    """
    将文本数据分割成重叠的块。
    
    参数:
        text_data (List[Dict]): 从 PDF 提取的文本数据
        chunk_size (int): 每个块的字符数大小
        overlap (int): 块之间的重叠字符数
        
    返回:
        List[Dict]: 分块的文本数据
    """
    chunked_data = []  # 初始化一个空列表来存储分块数据
    
    for item in text_data:
        text = item["content"]  # 提取文本内容
        metadata = item["metadata"]  # 提取元数据
        
        # 如果文本太短则跳过
        if len(text) < chunk_size / 2:
            chunked_data.append({
                "content": text,
                "metadata": metadata
            })
            continue
        
        # 创建带重叠的块
        chunks = []
        for i in range(0, len(text), chunk_size - overlap):
            chunk = text[i:i + chunk_size]  # 提取指定大小的块
            if chunk:  # 确保我们不添加空块
                chunks.append(chunk)
        
        # 添加每个块并更新元数据
        for i, chunk in enumerate(chunks):
            chunk_metadata = metadata.copy()  # 复制原始元数据
            chunk_metadata["chunk_index"] = i  # 向元数据添加块索引
            chunk_metadata["chunk_count"] = len(chunks)  # 向元数据添加总块数
            
            chunked_data.append({
                "content": chunk,  # 块文本
                "metadata": chunk_metadata  # 更新的元数据
            })
    
    print(f"创建了 {len(chunked_data)} 个文本块")  # 打印创建的块数量
    return chunked_data  # 返回分块数据列表

## 使用 OpenAI Vision 进行图像描述

In [5]:
def encode_image(image_path):
    """
    将图像文件编码为 base64。
    
    参数:
        image_path (str): 图像文件的路径
        
    返回:
        str: Base64 编码的图像
    """
    # 以二进制读取模式打开图像文件
    with open(image_path, "rb") as image_file:
        # 读取图像文件并编码为 base64
        encoded_image = base64.b64encode(image_file.read())
        # 将 base64 字节解码为字符串并返回
        return encoded_image.decode('utf-8')

In [6]:
def generate_image_caption(image_path):
    """
    使用 OpenAI 的视觉能力为图像生成描述。
    
    参数:
        image_path (str): 图像文件的路径
        
    返回:
        str: 生成的描述
    """
    # 检查文件是否存在且是图像
    if not os.path.exists(image_path):
        return "错误：未找到图像文件"
    
    try:
        # 打开并验证图像
        Image.open(image_path)
        
        # 将图像编码为 base64
        base64_image = encode_image(image_path)
        
        # 创建 API 请求来生成描述
        response = client.chat.completions.create(
            model="llava-hf/llava-1.5-7b-hf", # 使用 llava-1.5-7b 模型
            messages=[
                {
                    "role": "system",
                    "content": "你是一个专门描述学术论文中图像的助手。"
                    "为图像提供详细的描述，捕捉关键信息。"
                    "如果图像包含图表、表格或图形，请清楚地描述它们的内容和目的。"
                    "你的描述应该针对未来检索进行优化，当人们询问此内容时。"
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "详细描述这张图像，重点关注其学术内容："},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ],
            max_tokens=300
        )
        
        # 从响应中提取描述
        caption = response.choices[0].message.content
        return caption
    
    except Exception as e:
        # 如果发生异常，返回错误消息
        return f"生成描述时出错：{str(e)}"

In [7]:
def process_images(image_paths):
    """
    处理所有图像并生成描述。
    
    参数:
        image_paths (List[Dict]): 提取的图像路径
        
    返回:
        List[Dict]: 带有描述的图像数据
    """
    image_data = []  # 初始化一个空列表来存储带描述的图像数据
    
    print(f"为 {len(image_paths)} 张图像生成描述...")  # 打印要处理的图像数量
    for i, img_item in enumerate(image_paths):
        print(f"处理图像 {i+1}/{len(image_paths)}...")  # 打印当前正在处理的图像
        img_path = img_item["path"]  # 获取图像路径
        metadata = img_item["metadata"]  # 获取图像元数据
        
        # 为图像生成描述
        caption = generate_image_caption(img_path)
        
        # 将带描述的图像数据添加到列表中
        image_data.append({
            "content": caption,  # 生成的描述
            "metadata": metadata,  # 图像元数据
            "image_path": img_path  # 图像路径
        })
    
    return image_data  # 返回带描述的图像数据列表

## 简单向量存储实现

In [8]:
class MultiModalVectorStore:
    """
    多模态内容的简单向量存储实现。
    """
    def __init__(self):
        # 初始化列表来存储向量、内容和元数据
        self.vectors = []
        self.contents = []
        self.metadata = []
    
    def add_item(self, content, embedding, metadata=None):
        """
        向向量存储中添加一个项目。
        
        参数:
            content (str): 内容（文本或图像描述）
            embedding (List[float]): 嵌入向量
            metadata (Dict, optional): 附加元数据
        """
        # 将嵌入向量、内容和元数据追加到各自的列表中
        self.vectors.append(np.array(embedding))
        self.contents.append(content)
        self.metadata.append(metadata or {})
    
    def add_items(self, items, embeddings):
        """
        向向量存储中添加多个项目。
        
        参数:
            items (List[Dict]): 内容项目列表
            embeddings (List[List[float]]): 嵌入向量列表
        """
        # 遍历项目和嵌入向量，将每个添加到向量存储中
        for item, embedding in zip(items, embeddings):
            self.add_item(
                content=item["content"],
                embedding=embedding,
                metadata=item.get("metadata", {})
            )
    
    def similarity_search(self, query_embedding, k=5):
        """
        查找与查询嵌入向量最相似的项目。
        
        参数:
            query_embedding (List[float]): 查询嵌入向量
            k (int): 返回结果的数量
            
        返回:
            List[Dict]: 前 k 个最相似的项目
        """
        # 如果存储中没有向量，返回空列表
        if not self.vectors:
            return []
        
        # 将查询嵌入向量转换为 numpy 数组
        query_vector = np.array(query_embedding)
        
        # 使用余弦相似度计算相似性
        similarities = []
        for i, vector in enumerate(self.vectors):
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity))
        
        # 按相似度降序排序
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        # 返回前 k 个结果
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "content": self.contents[idx],
                "metadata": self.metadata[idx],
                "similarity": float(score)  # 转换为 float 以便 JSON 序列化
            })
        
        return results

## 创建嵌入向量

In [9]:
def create_embeddings(texts, model="BAAI/bge-en-icl"):
    """
    为给定的文本创建嵌入向量。
    
    参数:
        texts (List[str]): 输入文本
        model (str): 嵌入模型名称
        
    返回:
        List[List[float]]: 嵌入向量
    """
    # 处理空输入
    if not texts:
        return []
        
    # 如果需要，分批处理（OpenAI API 限制）
    batch_size = 100
    all_embeddings = []
    
    # 分批遍历输入文本
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]  # 获取当前批次的文本
        
        # 为当前批次创建嵌入向量
        response = client.embeddings.create(
            model=model,
            input=batch
        )
        
        # 从响应中提取嵌入向量
        batch_embeddings = [item.embedding for item in response.data]
        all_embeddings.extend(batch_embeddings)  # 将批次嵌入向量添加到列表中
    
    return all_embeddings  # 返回所有嵌入向量

## 完整的处理流水线

In [10]:
def process_document(pdf_path, chunk_size=1000, chunk_overlap=200):
    """
    为多模态 RAG 处理文档。
    
    参数:
        pdf_path (str): PDF 文件的路径
        chunk_size (int): 每个块的字符数大小
        chunk_overlap (int): 块之间的重叠字符数
        
    返回:
        Tuple[MultiModalVectorStore, Dict]: 向量存储和文档信息
    """
    # 为提取的图像创建目录
    image_dir = "extracted_images"
    os.makedirs(image_dir, exist_ok=True)
    
    # 从 PDF 提取文本和图像
    text_data, image_paths = extract_content_from_pdf(pdf_path, image_dir)
    
    # 对提取的文本进行分块
    chunked_text = chunk_text(text_data, chunk_size, chunk_overlap)
    
    # 处理提取的图像以生成描述
    image_data = process_images(image_paths)
    
    # 合并所有内容项目（文本块和图像描述）
    all_items = chunked_text + image_data
    
    # 提取内容用于嵌入
    contents = [item["content"] for item in all_items]
    
    # 为所有内容创建嵌入向量
    print("为所有内容创建嵌入向量...")
    embeddings = create_embeddings(contents)
    
    # 构建向量存储并添加项目及其嵌入向量
    vector_store = MultiModalVectorStore()
    vector_store.add_items(all_items, embeddings)
    
    # 准备包含文本块和图像描述计数的文档信息
    doc_info = {
        "text_count": len(chunked_text),
        "image_count": len(image_data),
        "total_items": len(all_items),
    }
    
    # 打印添加项目的摘要
    print(f"向向量存储添加了 {len(all_items)} 个项目（{len(chunked_text)} 个文本块，{len(image_data)} 个图像描述）")
    
    # 返回向量存储和文档信息
    return vector_store, doc_info

## 查询处理和响应生成

In [11]:
def query_multimodal_rag(query, vector_store, k=5):
    """
    查询多模态 RAG 系统。
    
    参数:
        query (str): 用户查询
        vector_store (MultiModalVectorStore): 包含文档内容的向量存储
        k (int): 要检索的结果数量
        
    返回:
        Dict: 查询结果和生成的响应
    """
    print(f"\n=== 处理查询：{query} ===\n")
    
    # 为查询生成嵌入向量
    query_embedding = create_embeddings(query)
    
    # 从向量存储中检索相关内容
    results = vector_store.similarity_search(query_embedding, k=k)
    
    # 分离文本和图像结果
    text_results = [r for r in results if r["metadata"].get("type") == "text"]
    image_results = [r for r in results if r["metadata"].get("type") == "image"]
    
    print(f"检索到 {len(results)} 个相关项目（{len(text_results)} 个文本，{len(image_results)} 个图像描述）")
    
    # 使用检索到的内容生成响应
    response = generate_response(query, results)
    
    return {
        "query": query,
        "results": results,
        "response": response,
        "text_results_count": len(text_results),
        "image_results_count": len(image_results)
    }

In [12]:

def generate_response(query, results):
    """
    基于查询和检索结果生成响应。
    
    参数:
        query (str): 用户查询
        results (List[Dict]): 检索到的内容
        
    返回:
        str: 生成的响应
    """
    # 从检索结果格式化上下文
    context = ""
    
    for i, result in enumerate(results):
        # 确定内容类型（文本或图像描述）
        content_type = "文本" if result["metadata"].get("type") == "text" else "图像描述"
        # 从元数据获取页码
        page_num = result["metadata"].get("page", "未知")
        
        # 将内容类型和页码追加到上下文中
        context += f"[来自第 {page_num} 页的{content_type}]\n"
        # 将实际内容追加到上下文中
        context += result["content"]
        context += "\n\n"
    
    # 指导 AI 助手的系统消息
    system_message = """你是一个专门回答包含文本和图像的文档问题的 AI 助手。
    你已经获得了文档中的相关文本段落和图像描述。
    使用这些信息为查询提供全面、准确的响应。
    如果信息来自图像或图表，请在答案中提及这一点。
    如果检索到的信息无法完全回答查询，请承认这些限制。"""

    # 包含查询和格式化上下文的用户消息
    user_message = f"""查询：{query}

    检索到的内容：
    {context}

    请基于检索到的内容回答查询。
    """
    
    # 使用 OpenAI API 生成响应
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ],
        temperature=0.1
    )
    
    # 返回生成的响应
    return response.choices[0].message.content

## 与纯文本 RAG 的评估对比

In [13]:
def build_text_only_store(pdf_path, chunk_size=1000, chunk_overlap=200):
    """
    构建纯文本向量存储用于比较。
    
    参数:
        pdf_path (str): PDF 文件的路径
        chunk_size (int): 每个块的字符数大小
        chunk_overlap (int): 块之间的重叠字符数
        
    返回:
        MultiModalVectorStore: 纯文本向量存储
    """
    # 从 PDF 提取文本（重用函数但忽略图像）
    text_data, _ = extract_content_from_pdf(pdf_path, None)
    
    # 分块文本
    chunked_text = chunk_text(text_data, chunk_size, chunk_overlap)
    
    # 提取内容用于嵌入
    contents = [item["content"] for item in chunked_text]
    
    # 创建嵌入向量
    print("为纯文本内容创建嵌入向量...")
    embeddings = create_embeddings(contents)
    
    # 构建向量存储
    vector_store = MultiModalVectorStore()
    vector_store.add_items(chunked_text, embeddings)
    
    print(f"向纯文本向量存储添加了 {len(chunked_text)} 个文本项目")
    return vector_store

In [14]:
def evaluate_multimodal_vs_textonly(pdf_path, test_queries, reference_answers=None):
    """
    比较多模态 RAG 与纯文本 RAG。
    
    参数:
        pdf_path (str): PDF 文件的路径
        test_queries (List[str]): 测试查询
        reference_answers (List[str], optional): 参考答案
        
    返回:
        Dict: 评估结果
    """
    print("=== 评估多模态 RAG 与纯文本 RAG ===\n")
    
    # 为多模态 RAG 处理文档
    print("\n为多模态 RAG 处理文档...")
    mm_vector_store, mm_doc_info = process_document(pdf_path)
    
    # 构建纯文本存储
    print("\n为纯文本 RAG 处理文档...")
    text_vector_store = build_text_only_store(pdf_path)
    
    # 为每个查询运行评估
    results = []
    
    for i, query in enumerate(test_queries):
        print(f"\n\n=== 评估查询 {i+1}：{query} ===")
        
        # 如果可用，获取参考答案
        reference = None
        if reference_answers and i < len(reference_answers):
            reference = reference_answers[i]
        
        # 运行多模态 RAG
        print("\n运行多模态 RAG...")
        mm_result = query_multimodal_rag(query, mm_vector_store)
        
        # 运行纯文本 RAG
        print("\n运行纯文本 RAG...")
        text_result = query_multimodal_rag(query, text_vector_store)
        
        # 比较响应
        comparison = compare_responses(query, mm_result["response"], text_result["response"], reference)
        
        # 添加到结果中
        results.append({
            "query": query,
            "multimodal_response": mm_result["response"],
            "textonly_response": text_result["response"],
            "multimodal_results": {
                "text_count": mm_result["text_results_count"],
                "image_count": mm_result["image_results_count"]
            },
            "reference_answer": reference,
            "comparison": comparison
        })
    
    # 生成总体分析
    overall_analysis = generate_overall_analysis(results)
    
    return {
        "results": results,
        "overall_analysis": overall_analysis,
        "multimodal_doc_info": mm_doc_info
    }

In [15]:
def compare_responses(query, mm_response, text_response, reference=None):
    """
    比较多模态和纯文本响应。
    
    参数:
        query (str): 用户查询
        mm_response (str): 多模态响应
        text_response (str): 纯文本响应
        reference (str, optional): 参考答案
        
    返回:
        str: 比较分析
    """
    # 评估员的系统提示词
    system_prompt = """你是比较两个 RAG 系统的专家评估员：
    1. 多模态 RAG：从文本和图像描述中检索
    2. 纯文本 RAG：仅从文本中检索

    基于以下标准评估哪个响应更好地回答了查询：
    - 准确性和正确性
    - 信息的完整性
    - 与查询的相关性
    - 来自视觉元素的独特信息（对于多模态）"""

    # 包含查询和响应的用户提示词
    user_prompt = f"""查询：{query}

    多模态 RAG 响应：
    {mm_response}

    纯文本 RAG 响应：
    {text_response}
    """

    if reference:
        user_prompt += f"""
    参考答案：
    {reference}
    """

        user_prompt += """
    比较这些响应并解释哪一个更好地回答了查询以及原因。
    注意多模态响应中来自图像的任何特定信息。
    """

    # 使用 meta-llama/Llama-3.2-3B-Instruct 生成比较
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    
    return response.choices[0].message.content

In [16]:
def generate_overall_analysis(results):
    """
    生成多模态与纯文本 RAG 的总体分析。
    
    参数:
        results (List[Dict]): 每个查询的评估结果
        
    返回:
        str: 总体分析
    """
    # 评估员的系统提示词
    system_prompt = """你是 RAG 系统的专家评估员。基于多个测试查询，
    提供比较多模态 RAG（文本 + 图像）与纯文本 RAG 的总体分析。

    重点关注：
    1. 多模态 RAG 优于纯文本的查询类型
    2. 纳入图像信息的具体优势
    3. 多模态方法的任何缺点或限制
    4. 何时使用每种方法的总体建议"""

    # 创建评估摘要
    evaluations_summary = ""
    for i, result in enumerate(results):
        evaluations_summary += f"查询 {i+1}：{result['query']}\n"
        evaluations_summary += f"多模态检索到 {result['multimodal_results']['text_count']} 个文本块和 {result['multimodal_results']['image_count']} 个图像描述\n"
        evaluations_summary += f"比较摘要：{result['comparison'][:200]}...\n\n"

    # 包含评估摘要的用户提示词
    user_prompt = f"""基于以下对 {len(results)} 个查询的多模态与纯文本 RAG 评估，
    提供比较这两种方法的总体分析：

    {evaluations_summary}

    请提供多模态 RAG 相对于纯文本 RAG 的相对优缺点的全面分析，
    特别关注图像信息如何贡献（或未贡献）响应质量。"""

    # 使用 meta-llama/Llama-3.2-3B-Instruct 生成总体分析
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    
    return response.choices[0].message.content

## 多模态 RAG 与纯文本 RAG 的评估

In [17]:
# 你的 PDF 文档路径
pdf_path = "data/attention_is_all_you_need.pdf"

# 定义针对文本和视觉内容的测试查询
test_queries = [
    "Transformer（基础模型）的 BLEU 分数是多少？",
]

# 用于评估的可选参考答案
reference_answers = [
    "Transformer（基础模型）在 WMT 2014 英德翻译任务上达到了 27.3 的 BLEU 分数，在 WMT 2014 英法翻译任务上达到了 38.1 的 BLEU 分数。",
]

# 运行评估
evaluation_results = evaluate_multimodal_vs_textonly(
    pdf_path=pdf_path,
    test_queries=test_queries,
    reference_answers=reference_answers
)

# 打印总体分析
print("\n=== 总体分析 ===\n")
print(evaluation_results["overall_analysis"])

=== 评估多模态 RAG 与纯文本 RAG ===


为多模态 RAG 处理文档...
从 data/attention_is_all_you_need.pdf 提取内容...


提取了 15 个文本段落和 3 张图像
创建了 59 个文本块
为 3 张图像生成描述...
处理图像 1/3...
处理图像 2/3...
处理图像 3/3...
为所有内容创建嵌入向量...
向向量存储添加了 62 个项目（59 个文本块，3 个图像描述）

为纯文本 RAG 处理文档...
从 data/attention_is_all_you_need.pdf 提取内容...
提取了 15 个文本段落和 3 张图像
创建了 59 个文本块
为纯文本内容创建嵌入向量...
向纯文本向量存储添加了 59 个文本项目


=== 评估查询 1：Transformer（基础模型）的 BLEU 分数是多少？ ===

运行多模态 RAG...

=== 处理查询：Transformer（基础模型）的 BLEU 分数是多少？ ===



C:\Users\faree\AppData\Local\Temp\ipykernel_14692\2117883450.py:75: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  "similarity": float(score)  # Convert to float for JSON serialization


检索到 5 个相关项目（5 个文本，0 个图像描述）

运行纯文本 RAG...

=== 处理查询：Transformer（基础模型）的 BLEU 分数是多少？ ===

检索到 5 个相关项目（5 个文本，0 个图像描述）

=== 总体分析 ===

**多模态 RAG 与纯文本 RAG 的总体分析**

基于对单个查询的评估，我们可以得出以下关于多模态 RAG 相对于纯文本 RAG 的分析：

**多模态 RAG 优于纯文本的查询类型：**

1. **视觉数据查询**：当查询涉及图表、图形、表格或其他视觉元素中包含的信息时，多模态 RAG 具有明显优势。在这种情况下，图像描述可以提供纯文本中可能缺失的关键信息。

2. **复合信息查询**：需要结合文本和视觉信息来提供完整答案的查询，多模态方法可以提供更全面的响应。

**纳入图像信息的具体优势：**

1. **信息完整性**：图像描述可以补充文本信息，提供更完整的知识库。

2. **数据验证**：图像中的信息可以作为文本信息的验证或补充，提高答案的可靠性。

3. **上下文丰富性**：视觉元素可以提供额外的上下文，帮助更好地理解复杂概念。

**多模态方法的缺点或限制：**

1. **处理复杂性**：多模态系统需要额外的图像处理和描述生成步骤，增加了系统复杂性和计算成本。

2. **图像描述质量依赖**：系统的性能很大程度上依赖于图像描述的质量。如果图像描述不准确或不完整，可能会影响整体性能。

3. **检索效率**：在当前评估的查询中，多模态 RAG 检索到的图像描述为 0，这表明对于某些类型的查询，额外的图像处理可能不会带来实际益处。

**当前评估的具体观察：**

在评估的查询"Transformer（基础模型）的 BLEU 分数是多少？"中，多模态 RAG 检索到 5 个文本块和 0 个图像描述。这表明：

- 对于这个特定查询，相关信息主要存在于文本中
- 图像内容与查询的相关性较低
- 在这种情况下，多模态和纯文本方法可能产生相似的结果

**何时使用每种方法的建议：**

1. **使用多模态 RAG 的情况：**
   - 文档包含大量图表、表格、图形等视觉元素
   - 查询可能涉及视觉信息
   - 需要最全面的信息检索
   - 对准确性要求高于效率要求

2. **